# Determines bounding boxes for each sulcus

This notebook determines bounding box around a sulcus. It uses a supervised database, in which each sulcus has been manually labelled.

# Imports

In [ ]:
import sys
import os
import json
import tempfile
import colorado as cld

The following line permits to import deep_folding even if this notebook is executed from the notebooks subfolder (and no install has been launched):

 /notebooks/use_transform.ipynb  
 /deep_folding/__init__.py

In [ ]:
sys.path.append((os.path.abspath('../')))
import deep_folding
print((os.path.dirname(deep_folding.__file__)))

# User-specific variables

In [ ]:
sulcus = 'S.T.s.ter.asc.ant.'

In [ ]:
side = 'L'

We now assign path names and other user-specific variables.

The source directory is where the database lies. It contains the morphologist analysis subfolder ANALYSIS/3T_morphologist


In [ ]:
src_dir = os.path.join(os.getcwd(), '../data/source/supervised')
src_dir = os.path.abspath(src_dir)
print(("src_dir = " + src_dir))

The target directory tgt_dir is where the files will be saved

In [ ]:
tgt_dir = os.path.join(os.getcwd(), '../data/target/bbox')
tgt_dir = os.path.abspath(tgt_dir)
print(("tgt_dir = " + tgt_dir))

In [ ]:
ref_dir = os.path.join(os.getcwd(), '../data/reference/bbox')
ref_dir = os.path.abspath(ref_dir)
print(("ref_dir = " + ref_dir))

In [ ]:
print((sys.argv))

Gets the normlized SPM file to get voxel size inside the program

In [ ]:
norm_dir = os.path.join(os.getcwd(), '../data/source/unsupervised')
norm_dir = os.path.abspath(norm_dir)
sub_dir = "ANALYSIS/3T_morphologist/100206/t1mri/default_acquisition"

# Illustration of main program uses

### Using external calls

In [ ]:
!python ../deep_folding/anatomist_tools/mask.py --help

### By using the main function call

In [ ]:
from deep_folding.anatomist_tools import mask
print((mask.__file__))

In [ ]:
args = "--help"
argv = args.split(' ')

In [ ]:
mask.main(argv)

### By using the API function call

In [ ]:
mask.bounding_box(src_dir=src_dir,
                  tgt_dir=tgt_dir,
                  sulcus=sulcus,
                  side=side,
                  number_subjects=0)

# Test example

In [ ]:
unsupervised_dir = os.path.join(os.getcwd(), '../data/source/unsupervised')
reference_dir = os.path.join(os.getcwd(), '../data/reference')
skeleton_file = os.path.join(unsupervised_dir,
                            'ANALYSIS/3T_morphologist/100206/t1mri/default_acquisition/default_analysis/segmentation',
                            'Lskeleton_100206.nii.gz')

In [ ]:
_, _, vol_mask = mask.bounding_box(src_dir=src_dir,
                                    tgt_dir=tgt_dir,
                                    sulcus=sulcus,
                                    side=side,
                                    number_subjects=1,
                                    out_voxel_size=1,
                                    skeleton_file=skeleton_file)

In [ ]:
vol_mask.shape 

In [ ]:
from soma import aims
import moving_averages as ma

temp_dir = tempfile.mkdtemp()
mask_filename = f"{temp_dir}/mask.nii.gz"
aims.write(vol_mask, mask_filename)
bucket_filename = f"{temp_dir}/mask.bck"
cmd = f"AimsFileConvert -c Bucket -t VOID -e 1 -i {mask_filename} -o {bucket_filename}"
os.system(cmd)

# Displays bucket file
bucket, bucket_raw, dxyz, rot, tr = ma.load_bucket(bucket_filename)
m = cld.buket_to_mesh(bucket)
cld.draw(m)


In [ ]:
fig = cld.new_figure()
m = cld.buket_to_mesh(bucket)
m_raw = cld.buket_to_mesh(bucket_raw)

In [ ]:
bucket_raw

In [ ]:
#cld.draw(m,fig,color=[0,255,0])
cld.draw_numpy_bucket(bucket_raw)

# Mask test with more than 1 subject

In [ ]:
src_dir = "/host/neurospin/dico/data/bv_databases/human/pclean/all"

_, _, vol_mask = mask.bounding_box(src_dir=src_dir,
                                   tgt_dir=tgt_dir,
                                   sulcus=sulcus,
                                   side=side,
                                   number_subjects=10,
                                   out_voxel_size=1,
                                   skeleton_file=skeleton_file)

In [ ]:
temp_dir = tempfile.mkdtemp()
mask_filename = f"{temp_dir}/mask.nii.gz"
aims.write(vol_mask, mask_filename)
bucket_filename = f"{temp_dir}/mask.bck"
cmd = f"AimsFileConvert -c Bucket -t VOID -e 1 -i {mask_filename} -o {bucket_filename}"
os.system(cmd)

# Displays bucket file
bucket, bucket_raw, dxyz, rot, tr = ma.load_bucket(bucket_filename)
m = cld.buket_to_mesh(bucket)
cld.draw(m)

# Result analysis

Prints the list of files of the target directory

In [ ]:
import os
tgt_dir_side = os.path.join(tgt_dir, side)
print(('\n'.join(os.listdir(tgt_dir_side))))

Expected output (we read the bounding_box file from the reference directory):

In [ ]:
ref_dir_side = os.path.join(ref_dir, side)
ref_file = os.listdir(ref_dir_side)[0]
print("ref_file = ", ref_file, '\n')
with open(os.path.join(ref_dir_side,ref_file), 'r') as f:
    data_ref = json.load(f)
    print((json.dumps(data_ref, sort_keys=True, indent=4)))
    box_ref = {k: data_ref[k] for k in ['bbmin_voxel', 'bbmax_voxel']}

Obtained output (we read the bounding_box file from the target directory):

In [ ]:
tgt_file = os.listdir(tgt_dir_side)[0]
print("tgt_file = ", tgt_file, '\n')
with open(os.path.join(tgt_dir_side,tgt_file), 'r') as f:
    data_target = json.load(f)
    print((json.dumps(data_target, sort_keys=True, indent=4)))
    box_target = {k: data_ref[k] for k in ('bbmin_voxel', 'bbmax_voxel')}

In [ ]:
box_target == box_ref

In [ ]:
box_ref

In [ ]:
box_target